In [194]:
import numpy as np
import random
import math
import collections
import operator
import time
from tqdm.auto import tqdm
import progressbar

In [195]:
class InvasiveWeed():
    def __init__(self, pmax, maxiter, delta_cap, num_exceeded_delta):
        seed_array = []
        for i in range(100):
            x = random.uniform(0, 10)
            y = random.uniform(0, 10)
            seed_array.append((round(x, 3),round(y, 3)))


        delta = delta_cap + 0.01
        c = 0
        current_fitness_dict = {}
        self.current_fitness_dict = current_fitness_dict
        best_fitness_list =  []

    
        start_time = time.time()

        for i in range(1, maxiter+1):
            fitness_array = self.get_fitness(seed_array)
            fitness_dict = dict(zip(seed_array, fitness_array))
            sorted_fitness_dict = {k: v for k, v in sorted(fitness_dict.items(), key=lambda item: item[1])}

            normalized_fitness_dict = self.normalize_fitness(sorted_fitness_dict)
            seed_children_dict = self.eval_fitness(normalized_fitness_dict)

            fitness_sum = sum(fitness_array)

            best_fitness_list.append(sorted(fitness_array)[0])
            self.best_fitness_list = best_fitness_list



            for seed, fitness in fitness_dict.items():
                current_fitness_dict[seed] = fitness

            if len(current_fitness_dict) > pmax:
                    self.prune_seeds()



            seed_array = seed_array + self.generate_population(seed_children_dict)

            if len(best_fitness_list) > 5:
                delta = best_fitness_list[-1] - best_fitness_list[-2]
            else:
                delta = delta_cap + 0.01



            #print(f"Iteration: {i}\n")
            #print(f"\tSummed fitness: {fitness_sum}")
            #print(f"\tBest three fitness scores from this iteration: {sorted(fitness_array)[:3]}")
            #print(f"\tBest seed: {list(sorted_fitness_dict.items())[0]})")
            #print(f"\tNew seeds from this iteration based on fitness score: {get_new_seed_count(seed_children_dict)}")
            #print("\n\n")

            if delta < delta_cap:
                c += 1

            if c == num_exceeded_delta:
                #print(f"The difference between iterations was lower than the set delta value of {delta_cap}",
                #        "\nStopping before the set maximum number of iterations\n")
                break
                
            self.i = i



        end_time = time.time()
        runtime = end_time - start_time
        self.runtime = runtime
        
        #print(f"InvasiveWeed(pmax = {pmax}, maxiter = {maxiter}, delta_cap = {delta_cap}, num_exceeded_data = {num_exceeded_delta})")
        #print(f"Last iteration: {i}\n")
        #print(f"\tBest seed: {min(current_fitness_dict.items(), key=operator.itemgetter(1))[0]}")
        #print(f"\tBest fitness: {min(current_fitness_dict.items(), key=operator.itemgetter(1))[1]}")
        #print(f"\tBest three fitness scores: {best_fitness_list[::-1][:3]}")
        #print(f"\tTotal runtime: {runtime}\n")
        
    def generate_population(self,seed_children_dict):
        new_seeds = []
        for seed, nchildren in seed_children_dict.items():
            for _ in range(nchildren):
                x = seed[0]
                y = seed[1]
                new_x = random.gauss(x,1)
                new_y = random.gauss(y,1)

                while new_x < 0 or new_x > 10:
                    new_x=random.gauss(x,1)

                while new_y < 0 or new_y > 10:
                    new_y=random.gauss(y,1)

                new_seeds.append((new_x, new_y))

        return new_seeds
    
    def get_fitness(self,seed_array):
        fitness_array = []
        for seed in seed_array:
            x = seed[0]
            y = seed[1]
            fitness = ( x * math.sin(4*x) ) + ( 1.1 * y * math.sin( 2 * y) )
            fitness_array.append(fitness)
            
        return fitness_array
    
    def normalize_fitness(self,sorted_fitness_dict):
        lowest_fitness_element = list(sorted_fitness_dict.items())[0]
        if lowest_fitness_element:
            new_vals = []
            for val in sorted_fitness_dict.values():
                new_val = val - list(sorted_fitness_dict.values())[0]
                new_vals.append(new_val)
            sorted_fitness_dict = dict(zip(sorted_fitness_dict.keys(), new_vals))

        return sorted_fitness_dict

    def eval_fitness(self,normalized_fitness_dict):
        children_seed_list = []
        maximum = max(normalized_fitness_dict.values())
        for val in normalized_fitness_dict.values():
            num_seed = round( ( (val+0.000001) ) / maximum)
            children_seed_list.append(num_seed)

        seed_children_dict = dict(zip(normalized_fitness_dict.keys(), children_seed_list[::-1]))

        return seed_children_dict

    def get_new_seed_count(self,seed_children_dict):
        new_seeds_count = 0
        for seed, children in seed_children_dict.items():
            for i in range(children):
                new_seeds_count += children

        return new_seeds_count
    
    def prune_seeds(self):
        sorted_current_fitness_dict = {k: v for k, v in sorted(self.current_fitness_dict.items(), key=lambda item: item[1])}
        self.current_fitness_dict = dict(list(sorted_current_fitness_dict.items())[0:100]) 
        
    def return_best_fitness_(self):
        return self.best_fitness_list[::-1][0]
    
    def return_best_iteration_(self):
        return self.i
    
    def return_runtime_(self):
        return self.runtime
    
    def return_best_seed_(self):
        sorted_current_fitness_dict = {k: v for k, v in sorted(self.current_fitness_dict.items(), key=lambda item: item[1])}
        return list(sorted_current_fitness_dict.keys())[0]


In [201]:
def grid_search(params):
    c = 0
    grid_search_dict = {}
    runtimes = []
    param_no = np.prod([len(x) for x in params.values()])
    total = progressbar.ProgressBar(maxval=param_no)
    total.start()
    for pmax in params['pmax']:
        for maxiter in params['maxiter']:
            for delta_cap in params['delta_cap']:
                for num_exceeded_delta in params['num_exceeded_delta']:
                    model = InvasiveWeed(pmax, maxiter, delta_cap, num_exceeded_delta)
                    best_fitness_score = model.return_best_fitness_()
                    best_iteration = model.return_best_iteration_()
                    grid_search_dict[pmax, maxiter, delta_cap, num_exceeded_delta, best_iteration] = best_fitness_score
                    runtimes.append(model.return_runtime_())
                    total.update(c)
                    c+=1
                    
    total.finish()
    sorted_results_dict = {k: v for k, v in sorted(grid_search_dict.items(), key=lambda item: item[1])}   
    

    best_params = list(sorted_results_dict.keys())[0]
    best_fitness = list(sorted_results_dict.values())[0]
    
    print(f"\nGridsearch results:")
    print(f"\tBest model:")
    print(f"\t\tInvasiveWeed(pmax = {best_params[0]}, maxiter = {best_params[1]}, delta_cap = {best_params[2]}, num_exceeded_delta = {best_params[3]})\n")
    print(f"\tStopped at iteration: {best_params[4]}\n")
    print(f"\tTotal runtime: {sum(runtimes)}\n")
    print(f"\tBest fitness: {list(sorted_results_dict.values())[0]}")
    print(f"\tBest seed: {model.return_best_seed_()}")
    
    return best_params, best_fitness
                

In [202]:
params = {
    'pmax' : [10, 10**2, 10**3],
    'maxiter' : [10**1, 10**2],
    'delta_cap' : [10**7, 10**8, 10**9, 10**10],
    'num_exceeded_delta' : [5,6,7,8,9,10]
}

best_params, best_fitness = grid_search(params)

KeyboardInterrupt: 